In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# Uninstall packages from Kaggle base image that are not needed.
!pip uninstall -qy jupyterlab jupyterlab-lsp
# Install the google-genai SDK for this codelab.
!pip install -qU 'google-genai==1.7.0'

In [ ]:
from google import genai
from google.genai import types

from IPython.display import Markdown, HTML, display

genai.__version__

In [ ]:
import os
from google import genai
from google.genai import types
from IPython.display import Markdown, HTML, display

# Try to get the API key from environment variables
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# If API key is not set in the environment, prompt the user to enter it
if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = input("Please enter your Google API key: ")

if not GOOGLE_API_KEY:
    raise ValueError("Google API key not found. Please set the GOOGLE_API_KEY environment variable or provide it manually.")

# Now initialize the client with the API key
client = genai.Client(api_key=GOOGLE_API_KEY)


In [ ]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

In [ ]:
import os
from google import genai
from google.genai import types
from IPython.display import Markdown, display

# Try to get the API key from environment variables
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# If API key is not set in the environment, prompt the user to enter it
if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = input("Please enter your Google API key: ")

if not GOOGLE_API_KEY:
    raise ValueError("Google API key not found. Please set the GOOGLE_API_KEY environment variable or provide it manually.")

# Initialize the GenAI client with the API key
client = genai.Client(api_key=GOOGLE_API_KEY)

# Start a conversation loop
print("Chatbot started! Type 'quit' to exit.")
while True:
    # Ask the user for a question
    user_input = input("You: ")

    # Exit condition for the loop
    if user_input.lower() == 'quit':
        print("Exiting the chatbot. Goodbye!")
        break

    # Call the API to generate a response based on the user's input
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=user_input
    )

    # Display the response
    display(Markdown(response.text))


In [ ]:
import os
from google import genai
from google.genai import types
from IPython.display import Markdown, display

# Try to get the API key from environment variables
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# If API key is not set in the environment, prompt the user to enter it
if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = input("Please enter your Google API key: ")

if not GOOGLE_API_KEY:
    raise ValueError("Google API key not found. Please set the GOOGLE_API_KEY environment variable or provide it manually.")

# Initialize the GenAI client with the API key
client = genai.Client(api_key=GOOGLE_API_KEY)

# Define the configuration for search grounding
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)

def query_with_search_grounding(user_input):
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=user_input,
        config=config_with_search,
    )
    return response.candidates[0]

# Start a conversation loop
print("Chatbot started! Type 'quit' to exit.")
while True:
    # Ask the user for a question
    user_input = input("You: ")

    # Exit condition for the loop
    if user_input.lower() == 'quit':
        print("Exiting the chatbot. Goodbye!")
        break

    # Option to toggle between search grounding or not
    search_toggle = input("Would you like to enable search grounding? (yes/no): ").strip().lower()

    if search_toggle == 'yes':
        # Run query with search grounding
        response = query_with_search_grounding(user_input)
        # Access the correct response content when search grounding is enabled
        display(Markdown(response.content.parts[0].text))
    else:
        # Run query without search grounding
        response = client.models.generate_content(
            model='gemini-2.0-flash',
            contents=user_input
        )quit

        # Display the response
        display(Markdown(response.text))
